In [ ]:
%%writefile k-means100_GPU.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <algorithm>
#include <limits>
#include <cuda_runtime.h>

using namespace std;

// Константы и параметры
const int size_mass = 100;   // размерность массива
const int dim = 3;           // размерность пространства
const int k = 3;             // количество кластеров
const int max_iterations = 100;        // Максимальное количество итераций
const float epsilon = 0.0001;       // Пороговое значение для остановки

// Путь к файлу с данными
string file = "my_mass100.txt";



// Функция для считывания из файла данных с размерностью (100, 3)
void readFromFile(float* data, int size, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}

// Задание начальных центров на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim], maxValues[dim];
    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            centers[i * dim + j] = 0;
        }
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_size_mass;

// CUDA-ядро для заполнения кластеров
__global__ void clusterDistrCUDA(float* data, float* centers, float* distances, int* cluster_indexes) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Работа всех нитей
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((data[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        int min_index = 0;
        float min_distance = distances[point * d_k];

        for (int dist_id = 1; dist_id < d_k; dist_id++) {
            if (distances[point * d_k + dist_id] < min_distance) {
                min_distance = distances[point * d_k + dist_id];
                min_index = dist_id;
            }
        }

        cluster_indexes[point] = min_index;
    }
}

// CUDA-ядро для обнуления счётчика и центров
__global__ void nullCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Блок для одной нити, обнуление счётчика
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                centers[i * d_dim + j] = 0.0f;
            }
            count_cluster_i[i] = 0;
        }
    }
}

// CUDA-ядро для пересчёта центров
__global__ void recalculCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

// Блок для всех нитей
    if (point < d_size_mass) {
        int cluster_id = cluster_indexes[point];
        atomicAdd(&count_cluster_i[cluster_id], 1);
        for (int j = 0; j < d_dim; j++) {
            // если точка относится к кластеру, её координаты суммируются
            atomicAdd(&centers[cluster_id * d_dim + j], data[point * d_dim + j]);
        }
    }
}


__global__ void coordCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i){
      int point = blockIdx.x * blockDim.x + threadIdx.x;
    // блок для одной нити
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                if (count_cluster_i[i] != 0) {
                    // вычисление координаты
                    centers[i * d_dim + j] /= count_cluster_i[i];
                }
            }
        }
    }
}


bool converged(float* new_centers, float* old_centers, float epsilon, int k) {
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            if (fabs(new_centers[i * dim + j] - old_centers[i * dim + j]) >= epsilon) {
                return false;
            }
        }
    }
    return true;
}

void k_means(float* data, int* cluster_indexes, float* centers, float* oldCenters, float* distances, float* dev_data, float* dev_centers, float* dev_distances, int* dev_cluster_indexes, int* dev_count_cluster_i) {
    // Копирование констант на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация центров на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Сохранение начальных центров на CPU
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    int iteration = 0;
    while (iteration < max_iterations) {
        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass*dim + blockSize - 1) / blockSize;

        // запуск ядра для заполнения кластеров
        clusterDistrCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_cluster_indexes);
        cudaDeviceSynchronize();

        nullCentersCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // запуск ядра для пересчёта центров
        recalculCentersCUDA<<<gridSize, blockSize>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        coordCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // копирование пересчитанных центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // проверка сходимости
        if (iteration > 0 && converged(centers, oldCenters, epsilon, k)) {
            cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
            break;
        }

        // сохранение вычисленных центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }

    // Копирование меток для точек с GPU на CPU
    cudaMemcpy(cluster_indexes, dev_cluster_indexes, size_mass * sizeof(int), cudaMemcpyDeviceToHost);

    // Вывод итоговых центров
    cout << "Result centers:" << endl;
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            cout << centers[i * dim + j] << " ";
        }
        cout << endl;
    }
    cout << endl;

    // Вывод меток для точек
    cout << "Labels:" << endl;
    for (int i = 0; i < size_mass; i++) {
        cout << cluster_indexes[i] << " ";
    }
    cout << endl;
}



int main() {


    // Выделение памяти на CPU
    float* myArray = new float[size_mass * dim];
    float* centers = new float[k * dim];
    float* oldCenters = new float[k * dim];
    float* distances = new float[size_mass * k];
    int* cluster_indexes = new int[size_mass];

    // Выделение памяти на GPU
    float *dev_data, *dev_centers, *dev_distances;
    int *dev_cluster_indexes, *dev_count_cluster_i;

    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_cluster_indexes, size_mass * sizeof(int));
    cudaMalloc((void**)&dev_count_cluster_i, k * sizeof(int));

    // Считывание данных из файла и запись в массив
    readFromFile(myArray, size_mass, file);

    // Копирование данных на устройство
    cudaMemcpy(dev_data, myArray, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Запуск алгоритма
    cout << "K-means algorithm. Mass (100,3)." << endl << endl;
    k_means(myArray, cluster_indexes, centers, oldCenters, distances, dev_data, dev_centers, dev_distances, dev_cluster_indexes, dev_count_cluster_i);

    // Освобождение памяти на CPU
    delete[] myArray;
    delete[] centers;
    delete[] oldCenters;
    delete[] distances;
    delete[] cluster_indexes;

    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_centers);
    cudaFree(dev_distances);
    cudaFree(dev_cluster_indexes);
    cudaFree(dev_count_cluster_i);

    return 0;
}

Writing k-means100_GPU.cu


In [ ]:
!nvcc k-means100_GPU.cu -o k-means100_GPU
# компилляция

In [ ]:
# Запуск
!./k-means100_GPU

K-means algorithm. Mass (100,3).

The algorithm converged on iteration 4.

Result centers:
-0.135667 -0.093 -0.401667 
-3.09461 -3.26282 -2.80821 
3.10323 2.87613 3.18839 

Labels:
0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 


In [ ]:
%%writefile k-means10000_GPU.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <algorithm>
#include <limits>
#include <cuda_runtime.h>

using namespace std;

// Константы и параметры
const int size_mass = 10000;   // размерность массива
const int dim = 3;           // размерность пространства
const int k = 3;             // количество кластеров
const int max_iterations = 100;        // Максимальное количество итераций
const float epsilon = 0.0001;       // Пороговое значение для остановки

// Путь к файлу с данными
string file = "my_mass10000.txt";

// Функция для считывания из файла данных с размерностью (10000, 3)
void readFromFile(float* data, int size, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}

// Задание начальных центров на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim], maxValues[dim];
    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            centers[i * dim + j] = 0;
        }
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_size_mass;

// CUDA-ядро для заполнения кластеров
__global__ void clusterDistrCUDA(float* data, float* centers, float* distances, int* cluster_indexes) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Работа всех нитей
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((data[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        int min_index = 0;
        float min_distance = distances[point * d_k];

        for (int dist_id = 1; dist_id < d_k; dist_id++) {
            if (distances[point * d_k + dist_id] < min_distance) {
                min_distance = distances[point * d_k + dist_id];
                min_index = dist_id;
            }
        }

        cluster_indexes[point] = min_index;
    }
}

// CUDA-ядро для обнуления счётчика и центров
__global__ void nullCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Блок для одной нити, обнуление счётчика
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                centers[i * d_dim + j] = 0.0f;
            }
            count_cluster_i[i] = 0;
        }
    }
}

// CUDA-ядро для пересчёта центров
__global__ void recalculCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

// Блок для всех нитей
    if (point < d_size_mass) {
        int cluster_id = cluster_indexes[point];
        atomicAdd(&count_cluster_i[cluster_id], 1);
        for (int j = 0; j < d_dim; j++) {
            // если точка относится к кластеру, её координаты суммируются
            atomicAdd(&centers[cluster_id * d_dim + j], data[point * d_dim + j]);
        }
    }
}


__global__ void coordCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i){
      int point = blockIdx.x * blockDim.x + threadIdx.x;
    // блок для одной нити
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                if (count_cluster_i[i] != 0) {
                    // вычисление координаты
                    centers[i * d_dim + j] /= count_cluster_i[i];
                }
            }
        }
    }
}


bool converged(float* new_centers, float* old_centers, float epsilon, int k) {
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            if (fabs(new_centers[i * dim + j] - old_centers[i * dim + j]) >= epsilon) {
                return false;
            }
        }
    }
    return true;
}

void k_means(float* data, int* cluster_indexes, float* centers, float* oldCenters, float* distances, float* dev_data, float* dev_centers, float* dev_distances, int* dev_cluster_indexes, int* dev_count_cluster_i) {
    // Копирование констант на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация центров на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Сохранение начальных центров на CPU
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    int iteration = 0;
    while (iteration < max_iterations) {
        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass*dim + blockSize - 1) / blockSize;

        // запуск ядра для заполнения кластеров
        clusterDistrCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_cluster_indexes);
        cudaDeviceSynchronize();

        nullCentersCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // запуск ядра для пересчёта центров
        recalculCentersCUDA<<<gridSize, blockSize>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        coordCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // копирование пересчитанных центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // проверка сходимости
        if (iteration > 0 && converged(centers, oldCenters, epsilon, k)) {
            cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
            break;
        }

        // сохранение вычисленных центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }

    // Копирование меток для точек с GPU на CPU
    cudaMemcpy(cluster_indexes, dev_cluster_indexes, size_mass * sizeof(int), cudaMemcpyDeviceToHost);

    // Вывод итоговых центров
    cout << "Result centers:" << endl;
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            cout << centers[i * dim + j] << " ";
        }
        cout << endl;
    }
    cout << endl;

    // Вывод меток для точек
    // cout << "Labels:" << endl;
    // for (int i = 0; i < size_mass; i++) {
    //    cout << cluster_indexes[i] << " ";
    //}
    //cout << endl;
}



int main() {


    // Выделение памяти на CPU
    float* myArray = new float[size_mass * dim];
    float* centers = new float[k * dim];
    float* oldCenters = new float[k * dim];
    float* distances = new float[size_mass * k];
    int* cluster_indexes = new int[size_mass];


    // Выделение памяти на GPU
    float *dev_data, *dev_centers, *dev_distances;
    int *dev_cluster_indexes, *dev_count_cluster_i;


    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_cluster_indexes, size_mass * sizeof(int));
    cudaMalloc((void**)&dev_count_cluster_i, k * sizeof(int));


    // Считывание данных из файла и запись в массив
    readFromFile(myArray, size_mass, file);

    // Копирование данных на устройство
    cudaMemcpy(dev_data, myArray, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Запуск алгоритма
    cout << "K-means algorithm. Mass (10000,3)." << endl << endl;
    k_means(myArray, cluster_indexes, centers, oldCenters, distances, dev_data, dev_centers, dev_distances, dev_cluster_indexes, dev_count_cluster_i);


    // Освобождение памяти на CPU
    delete[] myArray;
    delete[] centers;
    delete[] oldCenters;
    delete[] distances;
    delete[] cluster_indexes;

    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_centers);
    cudaFree(dev_distances);
    cudaFree(dev_cluster_indexes);
    cudaFree(dev_count_cluster_i);

    return 0;
}

Overwriting k-means10000_GPU.cu


In [ ]:
# компилляция
!nvcc k-means10000_GPU.cu -o k-means10000_GPU

In [ ]:
# Запуск
!./k-means10000_GPU

K-means algorithm. Mass (10000,3).

The algorithm converged on iteration 9.

Result centers:
0.000891174 0.0180173 -0.00597129 
3.00641 2.97493 2.98367 
-3.03726 -2.99951 -2.97725 



In [ ]:
%%writefile k-means100000_GPU.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <algorithm>
#include <limits>
#include <cuda_runtime.h>

using namespace std;

// Константы и параметры
const int size_mass = 100000;   // размерность массива
const int dim = 3;           // размерность пространства
const int k = 3;             // количество кластеров
const int max_iterations = 100;        // Максимальное количество итераций
const float epsilon = 0.0001;       // Пороговое значение для остановки

// Путь к файлу с данными
string file = "my_mass100000.txt";



// Функция для считывания из файла данных с размерностью (100000, 3)
void readFromFile(float* data, int size, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}

// Задание начальных центров на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim], maxValues[dim];
    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            centers[i * dim + j] = 0;
        }
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_size_mass;

// CUDA-ядро для заполнения кластеров
__global__ void clusterDistrCUDA(float* data, float* centers, float* distances, int* cluster_indexes) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Работа всех нитей
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((data[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        int min_index = 0;
        float min_distance = distances[point * d_k];

        for (int dist_id = 1; dist_id < d_k; dist_id++) {
            if (distances[point * d_k + dist_id] < min_distance) {
                min_distance = distances[point * d_k + dist_id];
                min_index = dist_id;
            }
        }

        cluster_indexes[point] = min_index;
    }
}

// CUDA-ядро для обнуления счётчика и центров
__global__ void nullCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Блок для одной нити, обнуление счётчика
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                centers[i * d_dim + j] = 0.0f;
            }
            count_cluster_i[i] = 0;
        }
    }
}

// CUDA-ядро для пересчёта центров
__global__ void recalculCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

// Блок для всех нитей
    if (point < d_size_mass) {
        int cluster_id = cluster_indexes[point];
        atomicAdd(&count_cluster_i[cluster_id], 1);
        for (int j = 0; j < d_dim; j++) {
            // если точка относится к кластеру, её координаты суммируются
            atomicAdd(&centers[cluster_id * d_dim + j], data[point * d_dim + j]);
        }
    }
}


__global__ void coordCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i){
      int point = blockIdx.x * blockDim.x + threadIdx.x;
    // блок для одной нити
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                if (count_cluster_i[i] != 0) {
                    // вычисление координаты
                    centers[i * d_dim + j] /= count_cluster_i[i];
                }
            }
        }
    }
}


bool converged(float* new_centers, float* old_centers, float epsilon, int k) {
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            if (fabs(new_centers[i * dim + j] - old_centers[i * dim + j]) >= epsilon) {
                return false;
            }
        }
    }
    return true;
}

void k_means(float* data, int* cluster_indexes, float* centers, float* oldCenters, float* distances, float* dev_data, float* dev_centers, float* dev_distances, int* dev_cluster_indexes, int* dev_count_cluster_i) {
    // Копирование констант на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация центров на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Сохранение начальных центров на CPU
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    int iteration = 0;
    while (iteration < max_iterations) {
        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass*dim + blockSize - 1) / blockSize;

        // запуск ядра для заполнения кластеров
        clusterDistrCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_cluster_indexes);
        cudaDeviceSynchronize();

        nullCentersCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // запуск ядра для пересчёта центров
        recalculCentersCUDA<<<gridSize, blockSize>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        coordCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // копирование пересчитанных центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // проверка сходимости
        if (iteration > 0 && converged(centers, oldCenters, epsilon, k)) {
            cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
            break;
        }

        // сохранение вычисленных центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }

    // Копирование меток для точек с GPU на CPU
    cudaMemcpy(cluster_indexes, dev_cluster_indexes, size_mass * sizeof(int), cudaMemcpyDeviceToHost);

    // Вывод итоговых центров
    cout << "Result centers:" << endl;
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            cout << centers[i * dim + j] << " ";
        }
        cout << endl;
    }
    cout << endl;

}



int main() {


    // Выделение памяти на CPU
    float* myArray = new float[size_mass * dim];
    float* centers = new float[k * dim];
    float* oldCenters = new float[k * dim];
    float* distances = new float[size_mass * k];
    int* cluster_indexes = new int[size_mass];

    // Выделение памяти на GPU
    float *dev_data, *dev_centers, *dev_distances;
    int *dev_cluster_indexes, *dev_count_cluster_i;

    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_cluster_indexes, size_mass * sizeof(int));
    cudaMalloc((void**)&dev_count_cluster_i, k * sizeof(int));

    // Считывание данных из файла и запись в массив
    readFromFile(myArray, size_mass, file);

    // Копирование данных на устройство
    cudaMemcpy(dev_data, myArray, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Запуск алгоритма
    cout << "K-means algorithm. Mass (100000,3)." << endl << endl;
    k_means(myArray, cluster_indexes, centers, oldCenters, distances, dev_data, dev_centers, dev_distances, dev_cluster_indexes, dev_count_cluster_i);

    // Освобождение памяти на CPU
    delete[] myArray;
    delete[] centers;
    delete[] oldCenters;
    delete[] distances;
    delete[] cluster_indexes;

    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_centers);
    cudaFree(dev_distances);
    cudaFree(dev_cluster_indexes);
    cudaFree(dev_count_cluster_i);

    return 0;
}

Writing k-means100000_GPU.cu


In [ ]:
# компилляция
!nvcc k-means100000_GPU.cu -o k-means100000_GPU

# Запуск
!./k-means100000_GPU

K-means algorithm. Mass (100000,3).

The algorithm converged on iteration 7.

Result centers:
0.00754567 -0.00359486 0.00310872 
3.00439 2.9949 3.00139 
-3.00314 -3.00628 -3.00803 



In [ ]:
%%writefile k-means1000000_GPU.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <algorithm>
#include <limits>
#include <cuda_runtime.h>

using namespace std;



// Константы и параметры
const int size_mass = 1000000;   // размерность массива
const int dim = 3;           // размерность пространства
const int k = 3;             // количество кластеров
const int max_iterations = 100;        // Максимальное количество итераций
const float epsilon = 0.0001;       // Пороговое значение для остановки



// Функция для считывания из файлов данных с размерностью (1000000, 3)
void readFromFile(float* data, int size, int part_number) {
    string filename = "my_mass1000000_part_" + to_string(part_number) + ".txt";
    ifstream file(filename);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim + 0] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    }
    else {
        cerr << "Error opening file: " << filename << endl;
    }
}



// Задание начальных центров на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim], maxValues[dim];
    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            centers[i * dim + j] = 0;
        }
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_size_mass;



// CUDA-ядро для заполнения кластеров
__global__ void clusterDistrCUDA(float* data, float* centers, float* distances, int* cluster_indexes) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Работа всех нитей
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((data[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        int min_index = 0;
        float min_distance = distances[point * d_k];

        for (int dist_id = 1; dist_id < d_k; dist_id++) {
            if (distances[point * d_k + dist_id] < min_distance) {
                min_distance = distances[point * d_k + dist_id];
                min_index = dist_id;
            }
        }

        cluster_indexes[point] = min_index;
    }
}

// CUDA-ядро для обнуления счётчика и центров
__global__ void nullCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Блок для одной нити, обнуление счётчика
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                centers[i * d_dim + j] = 0.0f;
            }
            count_cluster_i[i] = 0;
        }
    }
}

// CUDA-ядро для пересчёта центров
__global__ void recalculCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

// Блок для всех нитей
    if (point < d_size_mass) {
        int cluster_id = cluster_indexes[point];
        atomicAdd(&count_cluster_i[cluster_id], 1);
        for (int j = 0; j < d_dim; j++) {
            // если точка относится к кластеру, её координаты суммируются
            atomicAdd(&centers[cluster_id * d_dim + j], data[point * d_dim + j]);
        }
    }
}


__global__ void coordCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i){
      int point = blockIdx.x * blockDim.x + threadIdx.x;
    // блок для одной нити
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                if (count_cluster_i[i] != 0) {
                    // вычисление координаты
                    centers[i * d_dim + j] /= count_cluster_i[i];
                }
            }
        }
    }
}


bool converged(float* new_centers, float* old_centers, float epsilon, int k) {
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            if (fabs(new_centers[i * dim + j] - old_centers[i * dim + j]) >= epsilon) {
                return false;
            }
        }
    }
    return true;
}

void k_means(float* data, int* cluster_indexes, float* centers, float* oldCenters, float* distances, float* dev_data, float* dev_centers, float* dev_distances, int* dev_cluster_indexes, int* dev_count_cluster_i) {
    // Копирование констант на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация центров на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Сохранение начальных центров на CPU
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    int iteration = 0;
    while (iteration < max_iterations) {
        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass*dim + blockSize - 1) / blockSize;

        // запуск ядра для заполнения кластеров
        clusterDistrCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_cluster_indexes);
        cudaDeviceSynchronize();

        nullCentersCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // запуск ядра для пересчёта центров
        recalculCentersCUDA<<<gridSize, blockSize>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        coordCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // копирование пересчитанных центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // проверка сходимости
        if (iteration > 0 && converged(centers, oldCenters, epsilon, k)) {
            cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
            break;
        }

        // сохранение вычисленных центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }

    // Копирование меток для точек с GPU на CPU
    cudaMemcpy(cluster_indexes, dev_cluster_indexes, size_mass * sizeof(int), cudaMemcpyDeviceToHost);

    // Вывод итоговых центров
    cout << "Result centers:" << endl;
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            cout << centers[i * dim + j] << " ";
        }
        cout << endl;
    }
    cout << endl;

}



int main() {


    // Выделение памяти на CPU
    float* myArray = new float[size_mass * dim];
    float* centers = new float[k * dim];
    float* oldCenters = new float[k * dim];
    float* distances = new float[size_mass * k];
    int* cluster_indexes = new int[size_mass];

    // Выделение памяти на GPU
    float *dev_data, *dev_centers, *dev_distances;
    int *dev_cluster_indexes, *dev_count_cluster_i;

    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_cluster_indexes, size_mass * sizeof(int));
    cudaMalloc((void**)&dev_count_cluster_i, k * sizeof(int));

    // Считывание данных из файла и запись в массив
    for (int i = 0; i <= 9; i++) {
        readFromFile(myArray + i * 100000, size_mass / 10, i);
    }

    // Копирование данных на устройство
    cudaMemcpy(dev_data, myArray, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Запуск алгоритма
    cout << "K-means algorithm. Mass (1000000, 3)." << endl << endl;
    k_means(myArray, cluster_indexes, centers, oldCenters, distances, dev_data, dev_centers, dev_distances, dev_cluster_indexes, dev_count_cluster_i);

    // Освобождение памяти на CPU
    delete[] myArray;
    delete[] centers;
    delete[] oldCenters;
    delete[] distances;
    delete[] cluster_indexes;

    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_centers);
    cudaFree(dev_distances);
    cudaFree(dev_cluster_indexes);
    cudaFree(dev_count_cluster_i);

    return 0;
}

Overwriting k-means1000000_GPU.cu


In [ ]:
# компилляция
!nvcc k-means1000000_GPU.cu -o k-means1000000_GPU

# Запуск
!./k-means1000000_GPU

K-means algorithm. Mass (1000000, 3).

The algorithm converged on iteration 5.

Result centers:
3.01088 3.00506 3.00622 
-3.00779 -3.00491 -3.00217 
-0.000254991 -0.000580432 -0.00102649 

